# MATH 300: Numerical Analysis I Recitation

## Instructor: Liam Doherty

## Meeting Time/Place: F 11:00AM, Curtis 453

## Office Hours: MRC Hours MR 5p-7p, W 12p-2p, or in Korman 282 by appointment

Anybody who wants to turn in paper copies of Homework 2 can do so today.

Before I get into problems for the day, there are two things that I would like to revisit. The first is regarding the Lagrange error bounds. Recall from Theorem 3.3 that $\newline \newline$
$$
|f(x) - P(x)| = \Big|\frac{f^{(n + 1)}(\xi)}{(n + 1)!}\prod_{k = 0}^{n}(x - x_{k})\Big|
\newline
$$
for some $\xi \in (x_{0}, x_{n})$. In class, when we had applied this bound, we had done it over the **whole interval**. What this amounts to is not only maximizing the derivative factor, but also the polynomial that it multiplies, i.e., $\newline \newline$
$$
|f(x) - P(x)| \leq \frac{1}{(n + 1)!}\max_{\xi \in (x_{0}, x_{1})}\Big|f^{(n + 1)}(\xi)\Big | \cdot \max_{x \in (x_{0}, x_{n})}\Big|\prod_{k = 0}^{n}(x - x_{k})\Big|.
\newline
$$
While this bound does indeed capture the maximum of the error of the whole interval, we may only want a point estimate of the error (as is the case in some of the homework problems). In this case, if we wanted an estimate of the error at $x = a$, for example, we would do $\newline \newline$
$$
|f(a) - P(a)| \leq \frac{1}{(n + 1)!}\max_{\xi \in (x_{0}, x_{1})}\Big|f^{(n + 1)}(\xi)\Big | \cdot \Big|\prod_{k = 0}^{n}(a - x_{k})\Big|.
\newline
$$
Notice that now, we will get a sharper error estimate, but it is **only valid at the point**.

The other thing I want to mention is about a few packages that are available in Julia. I'd like to comment that `Dierckx.jl` (that I showed last time) also has the ability to provide integrals of cubic splines:

In [ ]:
using Dierckx
x = [0.8, 1.]
y = [0.22363362, 0.65809197]
spl = Spline1D(x, y, k = 1)
integrate(spl, 0.8, 1.)

This is one way to do numerical approximation of integrals, but it is not widely used because of lacking error estimates for not much more (if any) computational efficiency compared to traditional quadrature (which we will see). It's also worth noting that because the spline is only defined between the nodes, you will experience weird behavior if you try to evaluate the spline outside of the interval the nodes contain, so be careful of that:

In [ ]:
println(spl(1.))
println(spl(100.))
println(integrate(spl, 0.8, 100.))

The other thing I want to show is `ForwardDiff`, which is a package that allows you to take derivatives of arbitrary functions using **forward mode automatic differentiation** (may be familiar to those interested in ML).

In [ ]:
using ForwardDiff
g(x) = sin(x)
dg(x) = ForwardDiff.derivative(g, x)

println("Actual derivative at 1: ", cos(1))
println("Derivative with ForwardDiff: ", dg(1))

Now, we'll get to some problems. We're covering Richardson's extrapolation and basics of quadrature.

# Problem 4.2.1b

Apply the extrapolation process described in Example 1 to determine $N_{3}(h)$, an approximation to $f'(x_{0})$, for the following function and step size. $\newline \newline$
$$
f(x) = x + e^{x}, \;\; x_{0} = 0.0, \;\; h = 0.4.
\newline
$$

## Solution: 

We expect to get something close to $2$, since $\newline \newline$
$$
f'(x_{0}) = 1 + e^{0} = 2.
\newline
$$
Equation (4.16) in section 4.2 of the textbook tells us that a recursive formula for computing $N_{j}(h)$ is $\newline \newline$
$$
N_{j}(h) = N_{j - 1}\Big(\frac{h}{2}\Big) + \frac{N_{j - 1}(h/2) - N_{j - 1}(h)}{4^{j - 1} - 1}
\newline
$$
for each $j = 2, 3, \dots$, and for $j = 1$, we just use our "usual" methods to make a low-order approximation (this particular formula is valid for quadratic schemes). To compute $N_{3}(h)$, we begin by simply computing $N_{1}(h)$ (here we use centered differences to obtain that quadratic order): $\newline \newline$
$$
f'(0.0) \approx N_{1}(0.4) = \frac{f(0.4) - f(-0.4)}{0.8} \approx 2.0269.
\newline
$$
This is already pretty close to our expected result (notice that it is about $\mathcal{O}(h^{2}) \asymp \mathcal{O}(10^{-2})$ away from our desired solution), but we can do better. To compute $N_{2}(h)$, we first need $N_{1}(h/2)$, which is $\newline \newline$
$$
f'(0.0) \approx N_{1}(0.2) = \frac{f(0.2) - f(-0.2)}{0.4} \approx 2.0067.
\newline
$$
Then, $\newline \newline$
$$
N_{2}(0.4) = N_{1}(0.2) + \frac{N_{1}(0.2) - N_{1}(0.4)}{3} \approx 1.999966.
\newline
$$
We can see how well this approximation is doing already, but let's continue. We need $N_{2}(h/2)$ to compute $N_{3}(h)$, and to compute $N_{2}(h/2)$, we need $N_{1}(h/4)$: $\newline \newline$
$$
N_{1}(0.1) = \frac{f(0.1) - f(-0.1)}{0.2} \approx 2.0017.
\newline
$$
Now we can compute $N_{2}(h/2) = N_{2}(0.2)$: $\newline \newline$
$$
N_{2}(0.2) = N_{1}(0.1) + \frac{N_{1}(0.1) - N_{1}(0.2)}{3} \approx 2.000033.
$$
Finally, we have what we need to compute $N_{3}(h) = N_{3}(0.4)$: $\newline \newline$
$$
N_{3}(0.4) = N_{2}(0.2) + \frac{N_{2}(0.2) - N_{2}(0.4)}{15} \approx 2.000000.
\newline
$$
We've obtained a very accurate solution at only a small cost!

# Problem 4.3.1a

Approximate the following integral with the trapezoidal rule. $\newline \newline$
$$
\int_{0.5}^{1}x^{4}dx
\newline
$$

## Solution: 

The book lists the "trapezoidal rule" as being $\newline \newline$
$$
\int_{a}^{b}f(x)dx \approx \frac{b - a}{2}(f(a) + f(b)),
\newline
$$
but we can do better. Indeed, instead of essentially using one interpolant over the whole interval, we can partition the interval, interpolate over each subinterval, and sum the resulting integrals. This produces the following "generalization" (which is what most people actually mean when they refer to this scheme in practice): $\newline \newline$
$$
\int_{a}^{b}f(x)dx \approx \frac{h}{2}\Big(f(a) + 2f(a + h) + 2f(a + 2h) + \dots + 2f(a + Nh) + f(b)\Big).
\newline
$$
Here, we assume uniform spacing, and that there are $N$ interior nodes $a + nh$, for $n \in \{1, \dots, N\}$.

So, instead of doing the boring thing (which would be to make the approximation $\newline \newline$
$$
\int_{0.5}^{1}x^{4}dx \approx \frac{1 - 0.5}{2}(0.5^{4} + 1^{4}) = 0.25 \cdot (1.0625) = 0.265625
\newline
$$
for our answer), we'll split up the interval with $h = 0.1$ to obtain $\newline \newline$
$$
\int_{0.5}^{1}x^{4}dx = \int_{0.5}^{0.6}x^{4}dx + \int_{0.6}^{0.7}x^{4}dx + \int_{0.7}^{0.8}x^{4}dx + \int_{0.8}^{0.9}x^{4}dx + \int_{0.9}^{1}x^{4}dx.
\newline
$$
Now, we make the approximation $\newline \newline$
$$
\begin{align*}
    \int_{0.5}^{1}x^{4}dx &\approx \frac{0.1}{2}(f(0.5) + 2f(0.6) + 2f(0.7) + 2f(0.8) + 2f(0.9) + f(1)) \\
    &\approx 0.05(0.0625 + 0.2592 + 0.4802 + 0.8192 + 1.3122 + 1) \\
    &\approx 0.1967.
\end{align*}
\newline
$$
Of course, we can integrate by hand to see which is the better approximation: $\newline \newline$
$$
\int_{0.5}^{1}x^{4}dx = \frac{x^{5}}{5}\Big|_{0.5}^{1} = 0.2 - \frac{0.5^{5}}{5} = 0.2 - 0.1 \cdot 0.5^{4} = 0.2 - 0.1 \cdot 0.0625 = 0.2 - 0.00625 = 0.19375.
\newline
$$
As expected, the approximation with the finer mesh is closer to the true solution than with the "one-step" mesh.

# Problem 4.1.29

Consider the function $\newline \newline$
$$
e(h) = \frac{\epsilon}{h} + \frac{h^{2}}{6}M,
\newline
$$
where $M$ is a bound on the third derivative of a function. Show that $e(h)$ has a minimum at $\sqrt[3]{3\epsilon/M}$.

## Solution: 

All we need to do is perform the standard optimization techniques from calculus. The derivative is $\newline \newline$
$$
e'(h) = -\frac{\epsilon}{h^{2}} + \frac{h}{3}M,
\newline
$$
which means that a critical point $h$ must satisfy $\newline \newline$
$$
\frac{\epsilon}{h^{2}} = \frac{h}{3}M.
\newline
$$
Solving for $h$, we find that $h = \sqrt[3]{3\epsilon/M}$ is a critical point. Further, it is indeed a minimum value since $\newline \newline$
$$
e''(h) = \frac{2\epsilon}{h^{3}} + \frac{M}{3} > 0
\newline
$$
when $\epsilon$, $h$ and $M$ are positive (of course, this is a reasonable assumption to make).

The reason for picking this exercise is obviously not for its mathematical content; rather, what this exercise shows us is that if we have a formula with an error term for a numerical scheme, we can optimize it in this way to find an optimal selection for our step size $h$.

# Problems 4.3.5(a) & 4.3.9(a)

The integral we consider is $\newline \newline$
$$
\int_{0.5}^{1}x^{4}dx.
\newline
$$
(This is intentionally chosen as the integral from last week's recitation). Approximate this integral with Simpson's rule and the Midpoint rule.

## Solution: 

Recall that the exact solution to this integral is $\newline \newline$
$$
\int_{0.5}^{1}x^{4}dx = 0.19375.
\newline
$$
The (non-composite) Simpson's rule makes the approximation $\newline \newline$
$$
\int_{x_{0}}^{x_{2}}f(x)dx = \frac{h}{3}[f(x_{0}) + 4f(x_{1}) + f(x_{2})] + \mathcal{O}(h^{5}).
\newline
$$
Neglecting the error term gives the approximation $\newline \newline$
$$
\begin{align*}
    \int_{0.5}^{1}x^{4}dx &\approx \frac{0.25}{3}[(0.5)^{4} + 4(0.75)^{4} + 1^{4}] \\
    &\approx 0.19401.
\end{align*}
\newline
$$
This is consistent with our error bounds as well, since $0.25^5 \approx 0.001$.

The Midpoint rule makes a simpler approximation: $\newline \newline$
$$
\int_{x_{-1}}^{x_{1}}f(x)dx = 2hf(x_{0}) + \mathcal{O}(h^{3})
\newline
$$
Applying this to our problem while neglecting the error term gives $\newline \newline$
$$
\begin{align*}
    \int_{0.5}^{1}x^{4}dx &\approx 2(0.25)(0.75)^{4} \\
    &\approx 0.1582.
\end{align*}
\newline
$$
It is clear that although there is less computational effort here, the approximation is not as good as the approximation for Simpson's rule.

# Problem 4.4.16(a)

In multivariable calculus and statistics courses it is shown that $\newline \newline$
$$
\int_{-\infty}^{\infty}\frac{1}{\sigma\sqrt{2\pi}}e^{-(1/2)(x/\sigma)^{2}}dx = 1,
\newline
$$
for any positive $\sigma$. The function $\newline \newline$
$$
f(x) = \frac{1}{\sigma\sqrt{2\pi}}e^{-(1/2)(x/\sigma)^{2}}
\newline
$$
is the _normal density function with mean_ $\mu = 0$ _and standard deviation_ $\sigma$. The probability that a randomly chosen value described by this distribution lies in $[a, b]$ is given by $\int_{a}^{b}f(x)dx$. Approximate to within $10^{-5}$ the probability that a randomly chosen value described by this distribution will lie in $[-\sigma, \sigma]$.

## Solution: 

The integral we need to evaluate is $\newline \newline$
$$
\int_{-\sigma}^{\sigma}\frac{1}{\sigma\sqrt{2\pi}}e^{-(1/2)(x/\sigma)^{2}}dx.
\newline
$$
Our error bound is $\newline \newline$
$$
|E(f)| \leq \frac{b - a}{180}h^{4}\max_{\mu}|f^{(4)}(\mu)| \leq \frac{\sigma}{90}\cdot\frac{3}{\sigma^{5}\sqrt{2\pi}}h^{4} = \frac{1}{30\sigma^{4}\sqrt{2\pi}}h^{4}.
\newline
$$
(See the appendix for associated derivative calulations.) This, however, is not useful to us, since the error bound is dependent on $\sigma$! What we can do, however, is rescale the integral with a substitution so that we are integrating the standard Gaussian PDF (that is, the PDF of a random variable distributed as $\mathcal{N}(0, 1)$). Let $y = x/\sigma$. Then the original integral is equivalent to $\newline \newline$
$$
\int_{-1}^{1}\frac{1}{\sqrt{2\pi}}e^{-\frac{y^{2}}{2}}dy.
\newline
$$
From the fruits of our labor in the appendix, setting $\sigma = 1$ allows us to recover that a bound on the fourth derivative for this standard Gaussian is $\newline \newline$
$$
|f^{(4)}(x)| \leq \frac{3}{\sqrt{2\pi}}.
\newline
$$
Then, our overall bound on the error is $\newline \newline$
$$
E(f) \leq \frac{1}{90}h^{4}\cdot\frac{3}{\sqrt{2\pi}} = \frac{h^{4}}{30\sqrt{2\pi}}.
\newline
$$
To ensure this is smaller than the indicated error of $10^{-5}$, we need $\newline \newline$
$$
\frac{h^{4}}{30\sqrt{2\pi}} < 10^{-5},
\newline
$$
and solving this for $h$ gives $h < 0.16559\dots$. Therefore, we can pick $h = 0.1$ and we'll be in the desired tolerance. Using this step size, Simpson's rule makes the approximation $\newline \newline$
$$
\begin{align*}
    \int_{-1}^{1}\frac{1}{\sqrt{2\pi}}e^{-\frac{y^{2}}{2}}dy &\approx \frac{0.1}{3}\Big(f(-1) + 4f(-0.9) + 2f(-0.8) + 4f(-0.7) + 2f(-0.6) + \dots + 2f(0.6) + 4f(0.7) + 2f(0.8) + 4f(0.9) + f(1)\Big) \\
    &\approx 0.6826900.
\end{align*}
\newline
$$
This means that we are predicting that $68.269\%$ of samples will be within a standard deviation of the mean. Let's see if that agrees with computing this integral through one of Julia's built-in packages:

In [ ]:
using QuadGK

f(x) = 1/√(2*pi)*exp(-x^2/2)
a = -1; b = 1
quadgk(f, a, b)

The QuadGK package returns the approximated value of the integral, and an estimate on the error bound. Here, our computed value is about $0.6286895$ accurate to within about $10^{-10}$, and we can see that our approximation is within $10^{-5}$ of that answer. 

# Appendix of Calculations for Problem 4.4.16

We need to maximize the fourth derivative of $\newline \newline$
$$
f(x) = \frac{1}{\sigma\sqrt{2\pi}}e^{-\frac{1}{2\sigma^{2}}x^{2}}.
\newline
$$
Repeated differentiation gives the first through fifth derivatives: $\newline \newline$
$$
\begin{align*}
    f'(x) &= \frac{-x}{\sigma^{3}\sqrt{2\pi}}e^{-\frac{1}{2\sigma^{2}}x^{2}} \\
    f''(x) &= \Bigg(\frac{-1}{\sigma^{3}\sqrt{2\pi}} + \frac{x^{2}}{\sigma^{5}\sqrt{2\pi}}\Bigg)e^{-\frac{1}{2\sigma^{2}}x^{2}} \\
    f'''(x) &= \Bigg(\frac{3x}{\sigma^{5}\sqrt{2\pi}} - \frac{x^{3}}{\sigma^{7}\sqrt{2\pi}}\Bigg)e^{-\frac{1}{2\sigma^{2}}x^{2}} \\
    f^{(4)}(x) &= \Bigg(\frac{3}{\sigma^{5}\sqrt{2\pi}} - \frac{6x^{2}}{\sigma^{7}\sqrt{2\pi}} + \frac{x^{4}}{\sigma^{9}\sqrt{2\pi}}\Bigg)e^{-\frac{1}{2\sigma^{2}}x^{2}} \\
    f^{(5)}(x) &= \Bigg(\frac{-15x}{\sigma^{7}\sqrt{2\pi}} + \frac{10x^{3}}{\sigma^{9}\sqrt{2\pi}} - \frac{x^{5}}{\sigma^{11}\sqrt{2\pi}}\Bigg)e^{-\frac{1}{2\sigma^{2}}x^{2}}
\end{align*}
\newline
$$
To maximize the fourth derivative, we need to find where the fifth derivative equals zero. Since the exponential term is never zero, we find that the critical points satisfy $\newline \newline$
$$
x\Big(x^{4} - 10\sigma^{2}x^{2} + 15\sigma^{4}\Big) = 0,
\newline
$$
which means that $x = 0$ is a solution and there are four other solutions. An application of the first derivative test shows that $x = 0$ and two of the solutions to $x^{4} - 10\sigma^{2}x^{2} + 15\sigma^{4}$ are maxima of $f^{(4)}(x)$, and plugging in the numbers and using the fact that $\sigma > 0$ tells us that $x = 0$ is the location of the global maximum of $f^{(4)}(x)$. Evaluating $f^{(4)}(0)$ gives $\newline \newline$
$$
f^{(4)}(0) = \frac{3}{\sigma^{5}\sqrt{2\pi}},
$$
which is the original $\sigma$-dependent upper bound used in the solution.